# Import thư viện

In [1]:
from selenium import webdriver
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from time import time
from warnings import warn
from datetime import datetime
from datetime import timedelta
from pytz import timezone
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import re
import os
import sys
import glob
import shutil
from pathlib import Path
import re

In [2]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='C:\chromedriver', options=options)
driver.implicitly_wait(0)

In [3]:
list_all_url_movie = pd.read_csv('C:/WINDOWS/system32/craw_web/Quản trị cơ sở dữ liệu/Rotten Tomatoes/URL/URL_movie.csv', squeeze=True).tolist()

# Thông tin phim

In [4]:
# Mở url
#url = 'https://www.rottentomatoes.com/m/beg1994'
url = 'https://www.rottentomatoes.com/m/antman'
driver.get(url)

In [5]:
# Đợi cho đến khi url load xong
element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
soup = BeautifulSoup(html_of_interest, 'lxml')

In [6]:
# Tiêu đề phim
title = soup.select('h1[class="scoreboard__title"]')[0].text
title

'Ant-Man'

In [7]:
# Điểm đánh giá của nhà phê bình
tomatometer = soup.select('score-board')[0].attrs['tomatometerscore']
tomatometer

'83'

In [8]:
# Điểm đánh giá của khán giả
audience_score = soup.select('score-board')[0].attrs['audiencescore']
audience_score

'85'

In [18]:
# Số lượt đánh giá của nhà phê bình
num_tomatometer = soup.select('a[slot="critics-count"]')[0].text.strip()
num_tomatometer

'339 Reviews'

In [10]:
# Số lượt đánh giá của khán giả
num_audience_score = soup.select('a[slot="audience-count"]')[0].text.strip()
num_audience_score

'\n                            100,000+ Ratings'

In [11]:
# The Tomatometer score
tomatometer_state = soup.select('score-board')[0].attrs['tomatometerstate']
tomatometer_state

'certified-fresh'

In [12]:
# The Audience Score
audience_state = soup.select('score-board')[0].attrs['audiencestate']
audience_state

'upright'

In [20]:
# Thông tin phim
list_name_info = soup.select('div[class="meta-label subtle"]')
list_info = soup.select('div[data-qa="movie-info-item-value"]')

for num_info in range(len(list_name_info)):
    name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
    print(name_info + info)

RatingPG-13 (Sci-Fi Action Violence)
GenreAction, Adventure, Fantasy, Comedy
Original LanguageEnglish
DirectorPeyton Reed
ProducerKevin Feige
WriterEdgar Wright, Joe Cornish, Adam McKay, Paul Rudd
Release Date (Theaters)Jul 17, 2015 wide
Release Date (Streaming)Dec 8, 2015
Box Office (Gross USA)$180.2M
Runtime1h 57m
DistributorWalt Disney
Sound MixDolby Atmos, Dolby Digital
View the collectionMarvel Cinematic Universe


In [14]:
# Tóm tắt nội dụng phim
synopsis = soup.select('div[id="movieSynopsis"]')[0].text.strip()
synopsis

'Forced out of his own company by former protégé Darren Cross, Dr. Hank Pym (Michael Douglas) recruits the talents of Scott Lang (Paul Rudd), a master thief just released from prison. Lang becomes Ant-Man, trained by Pym and armed with a suit that allows him to shrink in size, possess superhuman strength and control an army of ants. The miniature hero must use his new skills to prevent Cross, also known as Yellowjacket, from perfecting the same technology and using it as a weapon for evil.'

In [15]:
# Diễn viên
list_raw_cast = soup.select('span[class="characters subtle smaller"]')
list_cast = []

for c in list_raw_cast:
    list_cast.append(c.attrs['title'])

cast = ', '.join(list_cast)
cast

"Paul Rudd, Michael Douglas, Evangeline Lilly, Corey Stoll, Bobby Cannavale, Judy Greer, Abby Ryder Fortson, Michael Peña, David Dastmalchian, T.I., Wood Harris, Hayley Atwell, John Slattery, Martin Donovan, Garrett Morris, Gregg Turkington, Rod Hallett, Joe Chrest, Peyton Reed, Edgar Wright, Joe Cornish, Adam McKay, Paul Rudd, Kevin Feige, Louis D'Esposito, Alan Fine, Victoria Alonso, Michael Grillo, Stan Lee, Edgar Wright, Russell Carpenter, Dan Lebental, Colby Parker Jr., Christophe Beck, Shepherd Frankel, Sammy Sheldon, Sarah Halley Finn"

In [16]:
.

SyntaxError: invalid syntax (<ipython-input-16-a5d5b61aa8a6>, line 1)

# Reviews phim

### Reviews của nhà phê bình

In [ ]:
# Chuyển đến url review
driver.get(url + '/reviews')

element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

In [ ]:
# Check có review của nhà phê bình không ?
check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

if check_critic_review != '':
    list_critic_review = []
    
    for rv in soup_critic_review.select('p[class="review-text"]'):
        list_critic_review.append(rv.text)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element_by_css_selector('rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
            soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

            for rv in soup_critic_review.select('p[class="review-text"]'):
                list_critic_review.append(rv.text.replace('  ', ''))
            
        except:
            break
    critic_review = '<>'.join(list_critic_review)
    print(critic_review)
elif check_critic_review == '':
    critic_review = 'Không có review của nhà phê bình'
    print(critic_review)

### Review của khán giả

In [ ]:
# Chuyển đến url review của khán giả
driver.get(url + '/reviews?type=user')

element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

In [ ]:
# Check có review của khán giả không ?
check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')[0].text.strip()

if check_audience_review != '':
    list_audience_review = []
    
    for rv in soup_audience_review.select('p[data-qa="review-text"]'):
        list_audience_review.append(rv.text)
    
    while True:
        sleep(1)
        try:
            next_button = driver.find_element_by_css_selector('rt-button[class="js-prev-next-paging-next"]')
            driver.execute_script("arguments[0].click();", next_button)
            
            element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
            html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
            soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

            for rv in soup_audience_review.select('p[data-qa="review-text"]'):
                list_audience_review.append(rv.text)
            
        except:
            break
    audience_review = '<>'.join(list_audience_review)
    print(audience_review)
elif check_audience_review == '':
    audience_review = 'Không có review của khán giả'
    print(audience_review)